# M2 Data Science - Social and Graph Data Management 
###### -- Project -- Yacine Mokhtari

**Graph:** Social Graph (Undirected)<br>
**Name & link:** [``musae-twitch``](http://snap.stanford.edu/data/twitch-social-networks.html)<br>

Here, I chose to focus only on the FR dataset.

## I. Load dataset and required libraries

In [33]:
# Imports
import json
import numpy as np 
from tqdm import tqdm
import networkx as nx
from math import comb, pow
import plotly.express as px
from collections import Counter
import plotly.graph_objects as go
import networkx.algorithms.community as nx_comm
import matplotlib.pyplot as plt
# conda install -c plotly plotly=5.11.0 (https://plotly.com/python/getting-started/)



PATH = "./dataset/[Social Graph] FR twitch/"
CSV_FILENAME = "musae_FR_edges.csv"
# JSON_FILENAME = "RO_genres.json" # may not be relevent for this study

G = nx.read_edgelist(PATH+CSV_FILENAME, delimiter=',',nodetype=int)

# --- Output some results
print(f"Number of nodes: {len(G.nodes())}")
print(f"Number of edges: {len(G.edges())}")
print(f"Number of connected components (sub-graphs): {nx.number_connected_components(G)}")

Number of nodes: 6549
Number of edges: 112666
Number of connected components (sub-graphs): 1


In [34]:
# Computing the average degree
deg_seq = [G.degree(x) for x in nx.nodes(G)]

average_degree = sum(deg_seq)*1./len(deg_seq)
print(f"Average degree: {average_degree}")
print(f"Highest degree: {max(deg_seq)}\n\t|- #Nodes with deg >= {max(deg_seq)/2} ; {np.sum([np.array(deg_seq) >= max(deg_seq)/2])}")
print(f"Lowest degree: {min(deg_seq)}\n\t|- #Nodes : {np.sum([np.array(deg_seq) == min(deg_seq)])}")

Average degree: 34.40708505115285
Highest degree: 2040
	|- #Nodes with deg >= 1020.0 ; 11
Lowest degree: 1
	|- #Nodes : 247


In [35]:
seed = 42
pos = nx.spring_layout(G, seed=seed) 

In [36]:
import plotly.io as pio
pio.renderers.default = "browser"


def draw_graph(G, layout_pos):
    """
        Built following this example/tutorial:
            https://plotly.com/python/network-graphs/
    Args:
        G (networkx): Graph
        layout_pos (2D array): Position of nodes, obtained after selecting a certain layout (see cell above)
    """
    # ------- EDGE TREATMENT
    # Edge coordinates
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = layout_pos[edge[0]]
        x1, y1 = layout_pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
    # edge component
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    # ------- NODE TREATMENT
    # node coordinates
    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = layout_pos[node]
        node_x.append(x)
        node_y.append(y)
    # node component
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))
    # -------  COLOR Following adjacency
    node_adjacencies = []
    node_text = []
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append('Degree: '+str(len(adjacencies[1])))

    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text 

    # -------- Create network graph
    fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='<br>Deezer: RO dataset',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        text="Network Analysis Project<br><small><b>M2 Data Science:</b> Social and Graph Data Management class<br>2022-2023</small>",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002 ) ],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    return fig

# ------------ Plot graph
fig = draw_graph(G, pos)
fig.write_html("./[HTML] plots/network graph.html")

## II. Generate a random graph sharing the same properties

In [37]:
## Computing the value of p (in a random network having the same characteristics)
average_degree = float(sum(deg_seq))/float(len(deg_seq))
N = len(G.nodes())
p = average_degree / (N-1)
print(f"<k>: {average_degree}")
print(f"The value of p: {p}")

# -----------------------------------------------------------------------------------

## Generating a random graph using N and p values and plot it
G_rand  = nx.gnp_random_graph(N, p)
deg_seq_rand = [G_rand.degree(x) for x in nx.nodes(G_rand)]


<k>: 34.40708505115285
The value of p: 0.005254594540493715


## III. Plots

#### 1. Degrees

In [38]:
hist_fig = go.Figure(go.Histogram(
    x=deg_seq,
    bingroup=1,
    name="FR Twitch Graph",
    opacity=0.75))

hist_fig.add_trace(go.Histogram(
    x=deg_seq_rand,
    bingroup=1,
    name="Random Graph",
    opacity=0.75))

hist_fig.update_layout(
    title_text='Histogram of degrees', # title of plot
    xaxis_title_text='Degree Value', # xaxis label
    yaxis_title_text='Count', # yaxis label
    barmode="overlay",
    bargap=0.1 # gap between bars of adjacent location coordinates
)

# hist_fig.show()
hist_fig.write_html("./[HTML] plots/degrees.html")

### 2. Clustering coefficient 

In [39]:
### Clustering coefficient :
clustering_coeff = list(nx.clustering(G).values())
clustering_coeff_rand = list(nx.clustering(G_rand).values())
### Average clustering coefficient values
avg_clustering_coeff = nx.average_clustering(G)
avg_clustering_coeff_rand = nx.average_clustering(G_rand)

### Getting interesting values
bin_min  =  min(np.min(clustering_coeff), np.min(clustering_coeff_rand))
bin_max  = max(np.max(clustering_coeff), np.max(clustering_coeff_rand))
bin_size = 1.*(bin_max - bin_min)/100

### Plot the results in a histogram
hist_fig2 = go.Figure(go.Histogram(
    x=list(clustering_coeff),
    xbins=dict( # bins used for histogram
        start=bin_min,
        end=bin_max,
        size=bin_size
    ),
    name="FR Twitch Graph",
    opacity=0.75))

hist_fig2.add_trace(go.Histogram(
    x=list(clustering_coeff_rand),
    xbins=dict( # bins used for histogram
        start=bin_min,
        end=bin_max,
        size=bin_size
    ),
    name="Random Graph",
    opacity=0.75))

# vlines average clust coeff

hist_fig2.update_layout(
    title_text='<b>Histogram of clustering coefficient</b>', # title of plot
    xaxis_title_text='Values', # xaxis label
    yaxis_title_text='Count', # yaxis label
    barmode="overlay",
    bargap=0.1 # gap between bars of adjacent location coordinates
)

# hist_fig2.show()
hist_fig2.write_html("./[HTML] plots/clustering coefficient.html")

### 3. Distances

In [40]:
def distance_seq(G):
    """Function that takes a graph as an input and returns a counter of all the distances from all the nodes

    Args:
        G (networkx): a graph

    Returns:
        Counter: distances/without information about the (source, target) couples
    """
    c = Counter()
    for i in tqdm(G.nodes()):
        c.update(list(nx.single_source_shortest_path_length(G, i).values()))
    return c

### Compute the dist_seq 
dist_counter = distance_seq(G)
dist_counter_rand = distance_seq(G_rand)

### Useful values
x1 = list(dist_counter.keys())
y1 = list(dist_counter.values())
x2 = list(dist_counter_rand.keys())
y2 = list(dist_counter_rand.values())

### Plot the results in a histogram
hist_fig3 = go.Figure(go.Histogram(
    x=x1, y=y1,
    histfunc="sum",
    name="FR Twitch Graph",
    xbins=dict(
        start=min(x1),
        end=max(x1),
        size=1
    ),
    opacity=0.75)
)

hist_fig3.add_trace(go.Histogram(
    x=x2, y=y2,
    histfunc="sum",
    name="Random Graph",
    xbins=dict(
        start=min(x1),
        end=max(x1),
        size=1
    ),
    opacity=0.75)
)
# # vlines average clust coeff

hist_fig3.update_layout(
    title_text='<b>Histogram of distances</b>', # title of plot
    xaxis_title_text='Values (length of the paths)', # xaxis label
    yaxis_title_text='Count', # yaxis label
    barmode="overlay",
    bargap=0.1 # gap between bars of adjacent location coordinates
)

# hist_fig3.show()
hist_fig3.write_html("./[HTML] plots/distances.html")

100%|██████████| 6549/6549 [01:10<00:00, 92.45it/s] 


### 4. Degree correlation

#### 4.a (Ugly) degree correlation matrix

In [41]:
deg_dict = G.degree()
degree_correlation_mat = np.zeros((max(deg_seq),  max(deg_seq)))  # matrix k x k , s.t. k is  the degree

# filling the degree correlation matrix
for n in tqdm(G.nodes()):
    for v in G.neighbors(n):
        if degree_correlation_mat[deg_dict[n]-1, deg_dict[v]-1] <= 5:
            degree_correlation_mat[deg_dict[n]-1, deg_dict[v]-1]+=1
            degree_correlation_mat[deg_dict[v]-1, deg_dict[n]-1]+=1

fig = go.Figure(data=go.Heatmap(
                    z=degree_correlation_mat, colorscale = 'BuPu'))
# fig.show()
fig.write_html("./[HTML] plots/degree coefficient matrix.html")

100%|██████████| 6549/6549 [00:00<00:00, 10478.39it/s]


#### 4.b Pearson degree correlation
The pearson degree correlation $r \in [-1, 1]$ is an indicator about the type of the graph.<br>


In [42]:
r = nx.degree_pearson_correlation_coefficient(G)
print(f"The (Pearson) degree correlation is {r}")

The (Pearson) degree correlation is -0.1781505683934457


**Remark:**<br>
In this case, we have $r = -0.17 \lt 0$ which means that this graph is an Assortative Network.
> Assortative networks : is a type of network graph in which nodes tend to connect to other nodes of similar degree.

### 5. Community detection

In [43]:
# list_communities  = nx_comm.greedy_modularity_communities(G)
print(f"Number of communities detection: {len(list_communities)}")
print(f"The average size of each community: {np.mean([len(x) for x in list_communities])}")
print(f"Size deviation : {np.std([len(x) for x in list_communities])} (pretty high)")
print(f"Maximum and minimum size: {np.max([len(x) for x in list_communities])}, {np.min([len(x) for x in list_communities])}")

Number of communities detection: 23
The average size of each community: 284.7391304347826
Size deviation : 715.6485788910707 (pretty high)
Maximum and minimum size: 2821, 2


### 6. Number of triangles
#### Expected number of triangles for random graphs
For three distinct nodes $i,j,k \in V$, the probability that the three edges $(i, j), (j, k)$ and $(i,k)$ exist is $p^3$.

Let $T_{i,j,k}$ a random variable that is equal to $1$ if the triangle on those three nodes exists in the graph $G$, or $0$ otherwise. (Hence, $\mathbb{P}(T_{i,j,k} = 1) = p^3$)

Then, the expected number of triangles in graph $G$ can be computed like this:
$$
\begin{aligned}
\mathbb{E}\left[\sum_{i,j,k}T_{i,j,k}\right] &= \sum_{i,j,k} \mathbb{E}\left[T_{i,j,k}\right] \\
&= \sum_{i,j,k \in V} \mathbb{P}\left(T_{i,j,k} = 1\right) \\
&= \binom n3 p^3

\end{aligned}
$$

In [44]:
### Computing our dataset's number of triangles
G_num_triangles = sum(nx.triangles(G).values())/3

### The random graph's expected number of triangles
G_rand_num_triangles = comb(N,3)*pow(p,3)

### Print and discuss
print(f"The number of triangles of our dataset is {G_num_triangles}")
print(f"The number of triangles of the random dataset is {G_rand_num_triangles}")

The number of triangles of our dataset is 422694.0
The number of triangles of the random dataset is 6788.790121529148


**Remarks:**
* The original value (from the karate graph) has a way far value of 422K.
* The expected value is not even the half of it, but this was expected!